In [ ]:
# obvykly uvodny tanec
import polars as pl
import plotly.express as px
import numpy as np  # potrebujeme pre histogram

In [ ]:
# nacitame vycistenu frejmu, stlpce, co pouzijeme v tomto NB, 
cols = ['pick_dt', 'drop_dt', 'distance']
df = pl.scan_parquet('data/nyc_taxi310k.parq')
df = df.select(cols).collect()

### V tomto NB nás budú zaujímať časy jázd a prejazdené vzdialenosti

In [ ]:
# vzdialenosti mame v stplci 'distance', ako ziskame casy?
(df['drop_dt'] - df['pick_dt']).head(6) #.cast(pl.Float32)/1000/60

In [ ]:
# pridame ako stlpec do df
df = df.with_columns(((pl.col('drop_dt') - pl.col('pick_dt')).cast(pl.Float32) / 1000 / 60).alias('rtimes'))
df.head()

In [ ]:
# na figu su nam teraz pick_dt a drop_dt, znicime ich
df_dt = df.drop(['pick_dt', 'drop_dt'])  # nie je to 'in_place'
df_dt.head()

In [ ]:
# predbezny prieskum -rozsahy
for col in df_dt.columns:
    print(col, df_dt[col].min(), df_dt[col].max())

In [ ]:
# ako vela je dlhych a dlhotrvajucich jazd? staci pocet
sd = df_dt.filter(pl.col('distance') > 10).shape # skusit 20, ...
st = df_dt.filter(pl.col('rtimes') > 30).shape # 120, 60, ...
print(sd, st)

In [ ]:
# na zaciatok pre vzd. horna hranica 10, pre casy 30
yd, xd = np.histogram(df_dt['distance'], bins=20, range=(0, 10)) # pochopit, co to vrati
yt, xt = np.histogram(df_dt['rtimes'], bins=20, range=(0, 30))
# np.histogram?

In [ ]:
# jedna funkcia pre kreslenie staci - aj s np.histogram vnutri
def plot_histo(col, rozsah, nbins=20):
    xlabel = 'Vzdialenosť (míle)' if col == 'distance' else 'Čas jazdy'
    y, x = np.histogram(df_dt[col], range=rozsah, bins=nbins)
    x = (x[0:-1] + x[1:]) / 2  # stredy intervalov, nie konce
    df_hist = pl.DataFrame({'x': x, 'y': y})  # pomocna frejma
    return px.bar(df_hist, x='x', y='y', barmode='group', 
                  labels={'x': xlabel, 'y': 'početnosť', 'variable': 'hodnota'}, width=900, height=350)

In [ ]:
plot_histo('rtimes', (0, 40), 40) # skusat 'distance', rozne nbins aj hornu hranicu
# preco sa pri niektorom pocte binov histogram pre vzdialenosti 'rozpada' ?
# df.filter(pl.col('distance') == pl.col('distance').round(1))

In [ ]:
# plot_histo('rtimes', (0, 35), 40) 

### Čo hovoria tie grafy? Že je to zrozumiteľnejšie, ako čísla vo frejmoch?

#### Pridáme do `datafunkcie.py` funkcie `plot_histo` aj `dt_frame` pre vyrobu datafrejmy s časmi jázd a vzdialenosťami 

In [ ]:
def dt_frame(frm): # frm bude povodna df, vyrobime df_dt
    df_dt = frm.with_columns(((pl.col('drop_dt') - pl.col('pick_dt')).cast(pl.Float32)/1000/60).alias('rtimes'))
    df_dt = df_dt.drop(['pick_dt', 'drop_dt'])
    return df_dt